# Задание №1. Разработка программного обеспечения для решения задачи линейного программирования

**Ф.И.О.:** Филиппов Владимир Леонидович    
**Поток:** МетОпт 1.2  
**Вариант:** 6

## Задача
Минимизировать:
\[
Z = 2x_1 + x_2 + x_3 + 3x_4
\]
при ограничениях:
\[
\begin{aligned}
x_1 + 2x_2 + x_4 &\le 10 \\
x_1 + x_3 + x_4 &= 7 \\
x_2 + 2x_3 &\ge 5 \\
\end{aligned}
\]

---

## Что делает программа:
1) читает постановку ЗЛП из **TXT-файла**;  
2) приводит задачу к **каноническому виду** (равенства + неотрицательность);  
3) строит **вспомогательную задачу** (фаза 1);  
4) решает фазу 1 симплекс-методом;  
5) если можно — переходит к фазе 2 и решает исходную задачу;  
6) печатает ответ или причину, почему решения нет.

**Входной файл:** `input.txt`.


## 1) Считывание постановки из TXT

Файл содержит только данные:

- 1-я строка: `min` или `max`
- 2-я строка: коэффициенты цели `c`
- далее ограничения:  
  `a1 a2 a3 a4 <= b`  (или `>=`, или `=`)

In [1]:
from pathlib import Path
import numpy as np

INPUT_FILE = "input.txt"

def read_lp_txt_nohash(path: str):
    # читаем все строки и убираем только пустые
    raw_lines = Path(path).read_text(encoding="utf-8").splitlines()
    lines = []
    for line in raw_lines:
        line = line.strip()
        if line != "":
            lines.append(line)

    if len(lines) < 3:
        raise ValueError("Нужно минимум 3 строки: min/max, c, и хотя бы одно ограничение.")

    sense = lines[0].lower()
    if sense not in ("min", "max"):
        raise ValueError("1-я строка должна быть min или max.")

    c = np.array([float(x) for x in lines[1].split()], dtype=float)

    A, signs, b = [], [], []
    for line in lines[2:]:
        parts = line.split()
        sign = parts[-2]
        rhs = float(parts[-1])
        coeffs = [float(x) for x in parts[:-2]]

        if len(coeffs) != len(c):
            raise ValueError("Неверное число коэффициентов в строке: " + line)
        if sign not in ("<=", ">=", "="):
            raise ValueError("Неверный знак в строке: " + line)

        A.append(coeffs); signs.append(sign); b.append(rhs)

    var_names = [f"x{i+1}" for i in range(len(c))]
    return sense, c, np.array(A, float), signs, np.array(b, float), var_names

sense, c, A, signs, b, var_names = read_lp_txt_nohash(INPUT_FILE)

print("Считали задачу из:", INPUT_FILE)
print("sense =", sense)
print("c =", c)
print("A=\n", A)
print("signs =", signs)
print("b =", b)
print("vars =", var_names)

Считали задачу из: input.txt
sense = min
c = [2. 1. 1. 3.]
A=
 [[1. 2. 0. 1.]
 [1. 0. 1. 1.]
 [0. 1. 2. 0.]]
signs = ['<=', '=', '>=']
b = [10.  7.  5.]
vars = ['x1', 'x2', 'x3', 'x4']


## 2) Приведение к каноническому виду

Симплекс-метод (в табличной форме) удобнее применять к задаче:

$$
\min c^T x \quad \text{при}\quad A_{eq}x=b,\; x\ge0
$$

Поэтому каждое ограничение приводим к равенству:

- Если `<=`: добавляем **добавочную** переменную (slack) `+s`  
  Пример: `a·x <= b` → `a·x + s = b`, `s>=0`
- Если `>=`: вычитаем **избыточную** `-s` и добавляем **искусственную** `+a`  
  `a·x >= b` → `a·x - s + a = b`, `s>=0, a>=0`
- Если `=`: добавляем **искусственную** `+a` (чтобы получить стартовый базис)  
  `a·x = b` → `a·x + a = b`, `a>=0`

Дальше используем **двухфазный метод**:
- **Фаза 1:** минимизируем сумму искусственных переменных `W = sum(a)`  
- **Фаза 2:** минимизируем исходную функцию `Z`


In [2]:
def to_canonical(A, b, signs, c, var_names):
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    m, n = A.shape

    names = list(var_names)   # имена всех переменных (будем добавлять)
    Aeq = A.copy()
    basis = []                # стартовый базис (индексы столбцов)
    artificial = []           # индексы искусственных переменных

    for i, sgn in enumerate(signs):
        if sgn == "<=":
            # + slack
            col = np.zeros((m, 1)); col[i, 0] = 1.0
            Aeq = np.hstack([Aeq, col])
            names.append(f"s{i+1}")
            basis.append(Aeq.shape[1]-1)

        elif sgn == ">=":
            # - surplus
            col_sur = np.zeros((m, 1)); col_sur[i, 0] = -1.0
            Aeq = np.hstack([Aeq, col_sur])
            names.append(f"s{i+1}")

            # + artificial
            col_art = np.zeros((m, 1)); col_art[i, 0] = 1.0
            Aeq = np.hstack([Aeq, col_art])
            names.append(f"a{i+1}")
            basis.append(Aeq.shape[1]-1)
            artificial.append(Aeq.shape[1]-1)

        elif sgn == "=":
            # + artificial
            col_art = np.zeros((m, 1)); col_art[i, 0] = 1.0
            Aeq = np.hstack([Aeq, col_art])
            names.append(f"a{i+1}")
            basis.append(Aeq.shape[1]-1)
            artificial.append(Aeq.shape[1]-1)

        else:
            raise ValueError("Неизвестный знак ограничения: " + str(sgn))

    # Цель фазы 2: исходная c на первых n, остальные 0
    c2 = np.zeros(Aeq.shape[1]); c2[:n] = c

    # Цель фазы 1: W = сумма искусственных
    c1 = np.zeros(Aeq.shape[1])
    for j in artificial:
        c1[j] = 1.0

    return Aeq, b, c1, c2, basis, names

Aeq, beq, c1, c2, basis, names = to_canonical(A, b, signs, c, var_names)

print("Все переменные:", names)
print("Размер Aeq:", Aeq.shape)
print("Aeq=\n", Aeq)
print("beq =", beq)
print("Стартовый базис (индексы) =", basis)

Все переменные: ['x1', 'x2', 'x3', 'x4', 's1', 'a2', 's3', 'a3']
Размер Aeq: (3, 8)
Aeq=
 [[ 1.  2.  0.  1.  1.  0.  0.  0.]
 [ 1.  0.  1.  1.  0.  1.  0.  0.]
 [ 0.  1.  2.  0.  0.  0. -1.  1.]]
beq = [10.  7.  5.]
Стартовый базис (индексы) = [4, 5, 7]


## 3) Симплекс-таблица и шаг pivot

Симплекс-таблица хранит:
- строки ограничений `Aeq | b`
- последнюю строку — коэффициенты цели

**Pivot (поворот)** — это элементарные преобразования строк, которые делают выбранный элемент ведущим (1),
а остальные элементы в этом столбце — 0.

Ниже — максимально простая реализация симплекса для **минимизации**:
- выбираем входящий столбец как самый отрицательный коэффициент в строке цели
- выбираем выходящую строку по правилу минимального отношения `b_i / a_i` для `a_i > 0`


In [3]:
def pivot(T, row, col):
    T = T.astype(float)
    T[row] = T[row] / T[row, col]
    for r in range(T.shape[0]):
        if r != row:
            T[r] = T[r] - T[r, col] * T[row]
    return T

def print_tableau(T, names=None, max_rows=20):
    # Печать таблицы в простом виде - вспомогательное 
    if names is None:
        print(T)
        return
    header = names + ["b"]
    print(" | ".join([f"{h:>8}" for h in header]))
    print("-" * (11 * len(header)))
    for i in range(min(T.shape[0], max_rows)):
        print(" | ".join([f"{T[i,j]:8.3f}" for j in range(T.shape[1])]))

def simplex_min(A, b, c, basis, max_iter=300, tol=1e-9, verbose=False, names=None):
    A = np.array(A, float)
    b = np.array(b, float)
    c = np.array(c, float)

    m, n = A.shape
    T = np.zeros((m+1, n+1))
    T[:m, :n] = A
    T[:m, -1] = b
    T[-1, :n] = c

    basis = list(basis)

    # Делаем строку цели канонической относительно текущего базиса
    for i, j in enumerate(basis):
        T[-1] -= T[-1, j] * T[i]

    if verbose:
        print("Начальная таблица:")
        print_tableau(T, names=names)

    for it in range(max_iter):
        col = int(np.argmin(T[-1, :n]))
        if T[-1, col] >= -tol:
            if verbose:
                print("Оптимум достигнут.")
            break

        ratios = np.full(m, np.inf)
        for i in range(m):
            if T[i, col] > tol:
                ratios[i] = T[i, -1] / T[i, col]

        row = int(np.argmin(ratios))
        if ratios[row] == np.inf:
            raise RuntimeError("unbounded")

        if verbose:
            print(f"Итерация {it+1}: pivot row={row}, col={col} (входит {names[col] if names else col})")

        T = pivot(T, row, col)
        basis[row] = col

        if verbose:
            print_tableau(T, names=names)

    x = np.zeros(n)
    for i, j in enumerate(basis):
        x[j] = T[i, -1]
    return x, T[-1, -1], basis

## 4) Двухфазный метод (полный цикл)

**Фаза 1:** решаем `min W = сумма искусственных`.  
Если `W_min > 0`, то исходная задача **несовместна** (нет допустимых решений).

**Фаза 2:** решаем исходную задачу `min Z`.  
Если в процессе получаем `unbounded`, то целевая функция **не ограничена**.

Для варианта 6 задача — `min`, поэтому мы решаем напрямую.
(Если бы было `max`, можно было бы заменить `max` на `min` умножением на `-1`.)


In [5]:
def solve_two_phase(Aeq, beq, c1, c2, basis, tol=1e-9, verbose=False, names=None):
    # Фаза 1
    try:
        _, W, basis1 = simplex_min(Aeq, beq, c1, basis, tol=tol, verbose=verbose, names=names)
    except RuntimeError:
        return {"status": "unbounded_phase1"}

    if W > tol:
        return {"status": "infeasible", "W": float(W)}

    # Фаза 2
    try:
        x, Z, _ = simplex_min(Aeq, beq, c2, basis1, tol=tol, verbose=verbose, names=names)
    except RuntimeError:
        return {"status": "unbounded"}

    return {"status": "optimal", "x": x, "Z": float(Z), "W": float(W)}

# verbose можно поставить True, чтобы печатать симплекс-таблицы
res = solve_two_phase(Aeq, beq, c1, c2, basis, verbose=False, names=names)
res

{'status': 'unbounded'}

## 5) Ответ в требуемом виде

Выводим:
- оптимальную точку `x* = (x1, x2, x3, x4)`
- значение `Z_min`

или сообщение о причине отсутствия решения.


In [6]:
if res["status"] == "optimal":
    x_main = res["x"][:len(var_names)]
    print("Оптимальная точка x*:")
    for i, nm in enumerate(var_names):
        print(f"{nm} = {x_main[i]:.6g}")
    print("\nZ_min =", f"{res['Z']:.6g}")
else:
    print("СТАТУС:", res["status"])
    if res["status"] == "infeasible":
        print("Причина: система ограничений несовместна (W_min > 0). W_min =", res["W"])
    elif res["status"] == "unbounded":
        print("Причина: целевая функция не ограничена снизу (unbounded).")

СТАТУС: unbounded
Причина: целевая функция не ограничена снизу (unbounded).


## 6) Проверка в Excel («Поиск решения»)
![Окно поиск решения](images/excel_solver_settings.png)
![Результат в таблице](images/excel_solver_result.png)

## 7) Вывод

В работе было полезно понять, почему симплекс-методу нужен стартовый базис и как его построить при ограничениях `=` и `>=`.
Двухфазный метод оказался удобным: фаза 1 проверяет совместность ограничений и находит допустимый базис, а фаза 2 уже оптимизирует исходную цель.
